In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()))

url = 'https://www.daraz.pk/'
driver.get(url)

ConnectionError: Could not reach host. Are you offline?

In [3]:
search = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search.click()

search.send_keys('Mobiles' + Keys.RETURN)

In [4]:

index, product, price, pRating, shipping, status, sRating, delievery, links = [] , [] , [] , [] , [] , [] , [] , [] , []
scroller, num, count = 100, 0, 0

for p in range(0,6):
    body = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]')
    adds = body.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    driver.execute_script(f"window.scrollBy({550}, 0);")

    for i in range(len(adds)):
        time.sleep(1)
        link = adds[i].find_element(By.TAG_NAME, 'a').get_attribute('href')
        links.append(link)

        Action = ActionChains(driver)
        Action.key_down(Keys.CONTROL).click(adds[i]).key_up(Keys.CONTROL).perform()
        driver.switch_to.window(driver.window_handles[1])

        info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span')
        product.append(info.text)

        try:
            info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span')
            price.append(info.text)
        except NoSuchElementException:
            price.append('0')

        try:
            info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
            sRating.append(info.text)
        except NoSuchElementException:
            sRating.append('0')

        try:
            info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]')
            ship = info.find_element(By.CLASS_NAME, 'seller-info-value ')
            delievery.append(ship.text)
        except NoSuchElementException:
            delievery.append('0')
        
        try:
            info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[2]')
            shipping.append('True')
        except NoSuchElementException:
            shipping.append('False')

        try:
            info = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/img')
            status.append('True')
        except NoSuchElementException:
            status.append('False')

        driver.execute_script('window.scrollTo(0, 400);')

        try:
            template = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]')
            rating = template.find_element(By.CLASS_NAME, 'score')
            pRating.append(rating.text)
        except NoSuchElementException:
            pRating.append('0')

        driver.close()
        driver.switch_to.window(driver.window_handles[0])       
        driver.execute_script(f'window.scrollTo(0, {scroller});')
        time.sleep(1)

        count += 1
        num += 1

        if count == 4:
            scroller += 100
            count = 0

        index.append(num)

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    next = driver.find_element(By.CLASS_NAME,"pager--mb0Ws").find_element(By.CLASS_NAME,"ant-pagination-next")

    next.click()
    scroller = 0
    time.sleep(5)

AttributeError: 'NoneType' object has no attribute 'text'

In [44]:
data = {
    'Index' : index,
    'Title': product,
    'Price': price,
    'Rating': pRating,
    'Free Delivery': shipping,
    'Seller Rating': sRating,
    'On Time Delievery': delievery,
    'Daraz Mall': status,
    'Links': links
}
df = pd.DataFrame(data)
df

ValueError: All arrays must be of the same length

In [ ]:
productname=[]
Reviews=[]
for page in range(0,5):
    time.sleep(6)
    ad_grid = driver.find_element(By.CLASS_NAME, 'box--ujueT')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    for i in ad_cards:
        try:
            producttitle=i.find_element(By.CLASS_NAME,'title--wFj93')
            producttitleb=producttitle.find_element(By.TAG_NAME,'a')
            productlink=producttitleb.get_attribute('href')
            driver.execute_script("window.open(arguments[0], '_blank');", productlink)
            driver.switch_to.window(driver.window_handles[1])
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(500,1000);")
            try:
                review=[]
                reviewa=driver.find_element(By.CLASS_NAME,'review-new-wrapper')
                reviewb=reviewa.find_element(By.CLASS_NAME,'review-content')
                reviewc=reviewb.find_elements(By.CLASS_NAME,'review-item')
                for i in reviewc:
                        reviewd=i.find_element(By.CLASS_NAME,'review-content-sl')
                        review.append(reviewd.text)
            except:
                review.append('None')
            time.sleep(1)
            Reviews.append(review)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass
    if(page==0):
            nxt_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[3]')                                      
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    elif(page==1):
            nxt_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[4]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    elif(page==2):
            nxt_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[5]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    elif(page==3):
            nxt_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[6]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    elif(page==4 or page>4):
            driver.close()
            break


In [ ]:
import pandas as pd
df = pd.read_csv('data/final.csv')
brands_to_check = ['apple', 'samsung', 'vivo', 'xiaomi', 'infinix', 'nokia', 'oppo','vgo','tecno','realme','redmi','itel','oneplus','huawei','sony','google']

for index, row in df.iterrows():
    if row['Brand'].lower() == 'no brand':
        title = row['Product Name'].lower()

        for brand in brands_to_check:
            if brand in title:
                df.at[index, 'Brand'] = brand.capitalize()

df['Product Rating'] = pd.to_numeric(df['Product Rating'], errors='coerce')

df['Seller Rating'] = df['Seller Rating'].str.replace('New Seller', '0%')

df['Seller Rating'] = pd.to_numeric(df['Seller Rating'].str.rstrip('%'), errors='coerce')

df['Price'] = df['Price'].str.replace('Rs. ', '').str.replace(',', '').astype(int)

df.to_csv('daraz.csv',index=False)
